In [ ]:
import optuna
from optuna.trial import Trial
import optuna.visualization as vis
import numpy as np
import jax.numpy as jnp
import jax
import pandas as pd
import matplotlib.pyplot as plt
import pyrcn
from pyrcn.base.blocks import InputToNode, NodeToNode
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

from chex import Array, Scalar
from typing import Tuple

from esn import ESN
from utils import chunkify, plot_predictions

key = jax.random.PRNGKey(43)

# Data

In [ ]:
N_SAMPLES = 10000
SPLIT_RATIO = 0.8
HISTORY_LEN = 10
FORECAST_LEN = 1

# Load from csv file
mackey_data = jnp.squeeze(jnp.array(pd.read_csv("data/MackeyGlass_10k.csv", index_col=0)))
# mackey_data = jnp.linspace(-1, 1, n_samples)

# Split into train-test
split = int(SPLIT_RATIO * N_SAMPLES)
mackey_train = mackey_data[:split]
mackey_test = mackey_data[split:]

# Plot
plt.plot(jnp.arange(0, split, dtype=jnp.int32), mackey_train, label="train")
plt.plot(jnp.arange(split, N_SAMPLES, dtype=jnp.int32), mackey_test, label="test")
plt.legend()
plt.show()

# Split into input and output
X_train, Y_train = chunkify(mackey_train, HISTORY_LEN, FORECAST_LEN)
X_test, Y_test = chunkify(mackey_test, HISTORY_LEN, FORECAST_LEN)

X_train_np, Y_train_np = np.array(X_train), np.array(Y_train)
X_test_np, Y_test_np = np.array(X_test), np.array(Y_test)

print(f"X train-test shapes: {X_train.shape}\t{X_test.shape}")
print(f"Y train-test shapes: {Y_train.shape}\t{Y_test.shape}")

# My ESN

## Optuna Search 

In [ ]:
def objective(trial: Trial) -> Scalar:

    # Define hyperparameters
    hn = trial.suggest_int("hidden_nodes", 10, 1000, log=True)
    # si = trial.suggest_float("sparsity_in", 0.0, 1.0)
    # sn = trial.suggest_float("sparsity_node", 0.0, 1.0)
    # sr = trial.suggest_float("spectral_radius", 0.0, 1.0)
    l = trial.suggest_float("leakage", 0.0, 1.0)
    l2 = trial.suggest_float("l2_cost", 0.1, 1000.0, log=True)

    # Create the model
    my_ESN = ESN(
        jax.random.PRNGKey(72),
        hidden_nodes=hn,
        sparsity_in=1.0,
        sparsity_node=1.0,
        input_activation=lambda x: x,
        node_activation=jax.nn.relu,
        spectral_radius=1.0,
        leakage=l,
        l2_cost=l2
    )

    # Train and evaluate the model
    my_ESN.fit(X_train, Y_train)
    y_hat_train = my_ESN.predict(X_train)
    mse_train = mean_squared_error(Y_train, y_hat_train)

    return mse_train

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

In [ ]:
# Evaluating our Optuna Trial
print("Best trial:", study.best_trial.number)
print("Best accuracy:", study.best_trial.value)
print("Best hyperparameters:", study.best_params)

In [ ]:
optimization_history_plot = vis.plot_optimization_history(study, error_bar=True)
optimization_history_plot.show()

In [ ]:
# Plotting a Contour Plot in Optuna
contour_plot = vis.plot_contour(study, params=["hidden_nodes"])
contour_plot.show()

## Evaluation

In [ ]:
# Create the model
my_ESN = ESN(
    jax.random.PRNGKey(72),
    hidden_nodes=14,
    sparsity_in=1.0,
    sparsity_node=1.0,
    input_activation=lambda x: x,
    node_activation=jax.nn.relu,
    spectral_radius=1.0,
    leakage=0.7867086239533064,
    l2_cost=0.1800606845001689
)

# Train and evaluate the model
my_ESN.fit(X_train, Y_train)
y_hat_train = my_ESN.predict(X_train)
mse_train = mean_squared_error(Y_train, y_hat_train)
print(f"MSE train: {mse_train}")

In [ ]:
X_train.shape, Y_train.shape, y_hat_train.shape

In [ ]:
n_first = 1000
plot_predictions(X_train[:, :n_first], Y_train[:, :n_first], y_hat_train[:, :n_first], HISTORY_LEN, FORECAST_LEN)

# PyRCN

## Optuna Search

In [ ]:
def objective(trial: Trial) -> Scalar:

    # Define hyperparameters
    hn = trial.suggest_int("hidden_nodes", 10, 1000, log=True)
    # si = trial.suggest_float("sparsity_in", 0.0, 1.0)
    # sn = trial.suggest_float("sparsity_node", 0.0, 1.0)
    # sr = trial.suggest_float("spectral_radius", 0.0, 1.0)
    l = trial.suggest_float("leakage", 0.0, 1.0)
    l2 = trial.suggest_float("l2_cost", 0.1, 1000.0, log=True)

    # Create the model
    input_to_node = InputToNode(
        hidden_layer_size=hn,
        k_in=None,
        sparsity=1.0,
        input_activation="identity",
    )
    node_to_node = NodeToNode(
        hidden_layer_size=hn,
        sparsity=1.0,
        reservoir_activation="tanh",
        spectral_radius=1.0,
        leakage=l,
        bidirectional=False
    )

    R_i2n = input_to_node.fit_transform(X_train_np.T)
    R_n2n = node_to_node.fit_transform(R_i2n)
    ridge_train = Ridge(alpha=l2).fit(R_n2n, np.squeeze(Y_train_np))

    # Train and evaluate the model
    y_hat_train_pyrcn = ridge_train.predict(R_n2n)
    mse_train_pyrcn = mean_squared_error(Y_train.T, y_hat_train_pyrcn)

    return mse_train_pyrcn

In [ ]:
study_pyrcn = optuna.create_study(direction="minimize")
study_pyrcn.optimize(objective, n_trials=100, timeout=10)

In [ ]:
# Evaluating our Optuna Trial
print("Best trial:", study_pyrcn.best_trial.number)
print("Best accuracy:", study_pyrcn.best_trial.value)
print("Best hyperparameters:", study_pyrcn.best_params)

In [ ]:
contour_plot = vis.plot_contour(study_pyrcn, params=["hidden_nodes", "leakage"])
contour_plot.show()